# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [3]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [4]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [6]:
model = Word2Vec(data, size=100, window=5, 
                        min_count=1, workers=4)


Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [7]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55564859, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [8]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [9]:
wv.most_similar('Texas')

[('Oregon', 0.8223920464515686),
 ('Ohio', 0.797393798828125),
 ('Pennsylvania', 0.7921767234802246),
 ('Arkansas', 0.7894842624664307),
 ('Florida', 0.7867615222930908),
 ('California', 0.7836043834686279),
 ('Louisiana', 0.7815113067626953),
 ('Massachusetts', 0.7781112194061279),
 ('Connecticut', 0.771111011505127),
 ('Utah', 0.7709168791770935)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [11]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.5000300407409668),
 ('Hunger-Free', 0.4125522971153259),
 ('UIUC', 0.4045395851135254),
 ('Rank-and-file', 0.3991539478302002),
 ('Headstrong', 0.39263916015625),
 ('Habitable', 0.39174842834472656),
 ('Trump-Voting', 0.3870272636413574),
 ('Likened', 0.3692036271095276),
 ('LaGrange', 0.3665298819541931),
 ('Compel', 0.35726073384284973)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [12]:
wv['Texas']

array([ 0.797582  , -2.2328155 ,  0.5028087 , -0.18995368,  0.20842603,
       -0.5846585 ,  1.5075827 , -0.2774331 , -0.3905989 ,  3.331426  ,
        0.7357984 ,  0.14374581, -1.9138222 ,  0.45052895, -2.0425382 ,
       -2.1240318 ,  0.7480771 ,  1.1813791 , -0.32739654, -1.5171695 ,
       -0.23625882,  0.72847867, -1.1466236 , -0.68264663,  0.14238971,
       -1.4211831 ,  0.06310856, -0.12326355,  2.5994275 ,  0.8598409 ,
       -2.0021164 ,  1.7600173 ,  0.05928344, -0.993198  , -1.176918  ,
        1.7042334 , -0.33687592,  1.2628485 , -2.6096716 ,  0.37247553,
       -1.2118723 , -0.7400199 , -0.3381532 , -0.26271877, -0.05866606,
        0.15429768,  1.7110904 , -2.3261368 ,  1.5789683 , -1.7019967 ,
        2.9558687 , -0.02187665, -1.1353866 , -1.6601912 , -0.5422774 ,
        0.3800766 ,  2.2710512 ,  1.9828233 , -0.20836072,  2.0818527 ,
        2.4832168 , -1.2619553 , -1.122569  ,  0.7358955 ,  0.55578727,
        2.159481  ,  0.22455004,  0.8906833 , -0.64468527,  0.93

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [13]:
wv.vectors

array([[-0.6431013 , -0.21893156, -0.05351847, ...,  0.7370132 ,
         0.21436998,  0.716481  ],
       [-0.1497582 , -1.0757767 ,  0.89696556, ...,  0.9870287 ,
         1.2709571 ,  1.1639824 ],
       [-1.0551136 ,  0.25114864,  1.0300958 , ...,  1.264914  ,
         0.92249906, -0.8056724 ],
       ...,
       [-0.05030591, -0.01256683,  0.01095608, ..., -0.06632469,
        -0.04842804, -0.00514463],
       [-0.0961495 ,  0.04157082,  0.02879434, ..., -0.02164887,
         0.00832067,  0.02313255],
       [-0.01029932, -0.02732416,  0.01674732, ...,  0.02903337,
        -0.07018884,  0.02620121]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [16]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('revival', 0.6491774320602417),
 ('princess', 0.6448976397514343),
 ('brunette', 0.6441299915313721),
 ('reminiscent', 0.6399538516998291),
 ('queen', 0.6211073398590088),
 ('villain', 0.613832414150238),
 ('symbol', 0.6059795618057251),
 ('crown', 0.5960112810134888),
 ('title', 0.5826130509376526),
 ('monster', 0.5795202851295471)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!